In [ ]:
# 📊 Benchmark: Next.js en Docker vs VM

Este notebook compara el rendimiento de una app Next.js ejecutada en Docker y en una Máquina Virtual (VM) usando pruebas de carga con `wrk` y monitoreo de recursos con `psutil`.

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import re

In [ ]:
# Cambia la ruta si tienes archivos separados para Docker y VM
df = pd.read_csv("../results/resource_usage.csv")

plt.figure(figsize=(10, 4))
plt.plot(df["timestamp"], df["cpu_percent"], label="CPU (%)")
plt.plot(df["timestamp"], df["mem_percent"], label="Memoria (%)")
plt.xlabel("Timestamp")
plt.ylabel("Uso (%)")
plt.title("Uso de CPU y Memoria durante el benchmark")
plt.legend()
plt.show()

In [ ]:
# Cambia la ruta si tienes archivos separados para Docker y VM
with open("../results/wrk_result.txt") as f:
    wrk_output = f.read()

print(wrk_output)

In [ ]:
def parse_wrk_output(output):
    metrics = {}
    # Requests/sec
    reqs = re.search(r"Requests/sec:\s+([\d\.]+)", output)
    if reqs:
        metrics["Requests/sec"] = float(reqs.group(1))
    # Latency
    latency = re.search(r"Latency\s+([\d\.]+)(ms|s|us)", output)
    if latency:
        value, unit = float(latency.group(1)), latency.group(2)
        # Convert latency to ms
        if unit == "s":
            value *= 1000
        elif unit == "us":
            value /= 1000
        metrics["Latency (ms)"] = value
    # Transfer/sec
    transfer = re.search(r"Transfer/sec:\s+([\d\.]+)(\w+)", output)
    if transfer:
        metrics["Transfer/sec"] = transfer.group(1) + transfer.group(2)
    return metrics

wrk_metrics = parse_wrk_output(wrk_output)
wrk_metrics

In [ ]:
# Comparar métricas de wrk entre Docker y VM

resultados = {}
for env in ["docker", "vm"]:
    with open(f"../results/wrk_result_{env}.txt") as f:
        resultados[env] = parse_wrk_output(f.read())

import pandas as pd
pd.DataFrame(resultados)